# Find proteins outside the 8p loss event (trans) that are commonly different between samples with and without the event

# Only looking at interacting proteins

## Setup

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
CHROMOSOME = "8"
ARM = "p"
TRANS_OR_CIS = "trans"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_filtered.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,brca_Database_ID,lscc_Database_ID,luad_Database_ID,ovarian_Database_ID,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,brca_diff,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,,,,
AAR2,NP_001258803.1,NP_001258803.1,NP_001258803.1,NP_001258803,0.772996,0.384689,0.921518,0.712527,0.864374,0.983639,0.167240,0.119473,-0.014591,0.083388,0.062387,0.039881
ABCE1,NP_001035809.1,NP_001035809.1,NP_001035809.1,NP_001035809,0.900647,0.996243,0.629196,0.062344,0.858032,0.806971,0.061418,0.003403,0.062801,0.312345,0.078355,-0.093025
ABCF3,NP_060828.2,NP_060828.2|NP_001338227.1|NP_001338229.1|NP_0...,NP_060828.2|NP_001338227.1|NP_001338229.1|NP_0...,NP_060828,0.783423,0.944775,0.210696,0.012419,0.902429,0.977919,0.166022,0.015720,0.147387,0.669350,0.045470,0.090238
ACLY,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001087,0.787837,0.345306,0.211365,0.106243,0.536111,0.911811,0.217653,0.123752,0.177086,0.357613,0.238336,-0.115389
ACLY,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001290204,0.787837,0.345306,0.211365,0.106243,0.536111,0.923814,0.217653,0.123752,0.177086,0.357613,0.238336,-0.113338
ACLY,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001290204.1|NP_942127.1,NP_001087,0.787837,0.345306,0.211365,0.106243,0.890306,0.911811,0.217653,0.123752,0.177086,0.357613,-0.100017,-0.115389
ACLY,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001290204.1|NP_942127.1,NP_001290204,0.787837,0.345306,0.211365,0.106243,0.890306,0.923814,0.217653,0.123752,0.177086,0.357613,-0.100017,-0.113338
ACLY,NP_001290203.1|NP_001087.2,NP_001290204.1|NP_942127.1,NP_001290203.1|NP_001087.2,NP_001087,0.787837,0.345306,0.211365,0.956326,0.536111,0.911811,0.217653,0.123752,0.177086,0.025934,0.238336,-0.115389
ACLY,NP_001290203.1|NP_001087.2,NP_001290204.1|NP_942127.1,NP_001290203.1|NP_001087.2,NP_001290204,0.787837,0.345306,0.211365,0.956326,0.536111,0.923814,0.217653,0.123752,0.177086,0.025934,0.238336,-0.113338


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,AAR2,NP_001258803.1,0.772996,0.167240
1,brca,ABCE1,NP_001035809.1,0.900647,0.061418
2,brca,ABCF3,NP_060828.2,0.783423,0.166022
3,brca,ACTB,NP_001092.1,0.952113,-0.055616
4,brca,AGAP1,NP_001032208.1|NP_055729.2,0.956263,-0.030548
5,brca,AGPS,NP_003650.1,0.899154,-0.213237
6,brca,AHCYL1,NP_006612.2|NP_001229602.1,0.972737,0.017116
7,brca,ALG2,NP_149078.1,0.972737,0.019412
8,brca,AP2B1,NP_001025177.1|NP_001273.1,0.984696,0.005864
9,brca,APP,NP_000475.1|NP_001191230.1|NP_001129602.1|NP_9...,0.899154,0.147745


## Select the proteins with a significant change

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)

In [7]:
prots.groupby("cancer_type").count()[["protein"]]

,protein
cancer_type,
brca,4
colon,8
hnscc,3
lscc,38
luad,13


## Find how many cancers each protein was different in

In [8]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist())
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,num_cancers
protein,,
ATP6V1H,"[brca, colon, lscc, luad]",4
ATP6V1E1,"[colon, hnscc, lscc, luad]",4
ATP6V1A,"[brca, colon, luad]",3
ATP6V1D,"[brca, luad]",2
PPP2R1A,"[brca, luad]",2
ATP6V1F,"[colon, luad]",2
MAST3,"[hnscc, lscc]",2
HSPA4L,[colon],1
NARS,[colon],1


In [9]:
prots_summary["num_cancers"].value_counts().sort_index(ascending=False)

4     2
3     1
2     4
1    47
Name: num_cancers, dtype: int64

## Save results

In [10]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}_filtered.tsv"
prots_summary.to_csv(output_file, sep="\t")